In [ ]:
import heapq
import sys


class OrderBook:
    def __init__(self):
        self.buy_orders = []
        self.sell_orders = []
        self.order_book = {}
        self.canceled_orders = set()
        self.best_bid = 0
        self.best_ask = 0

    def submit_order(self, side, order_id, quantity, price):
        if order_id in self.order_book or order_id in self.canceled_orders:
            return

        order = (price, order_id, quantity)
        self.order_book[order_id] = order

        if side == 'B':
            heapq.heappush(self.buy_orders, (-price, order_id, quantity))
            if price > self.best_bid:
                self.best_bid = price
        elif side == 'S':
            heapq.heappush(self.sell_orders, (price, order_id, quantity))
            if price < self.best_ask or self.best_ask == 0:
                self.best_ask = price

    def cancel_order(self, order_id):
        if order_id in self.order_book and order_id not in self.canceled_orders:
            order = self.order_book[order_id]
            self.canceled_orders.add(order_id)

            if order[0] == self.best_bid:
                self.update_best_bid()

            if order[0] == self.best_ask:
                self.update_best_ask()

    def update_best_bid(self):
        while self.buy_orders and self.buy_orders[0][1] in self.canceled_orders:
            heapq.heappop(self.buy_orders)
        self.best_bid = -self.buy_orders[0][0] if self.buy_orders else 0

    def update_best_ask(self):
        while self.sell_orders and self.sell_orders[0][1] in self.canceled_orders:
            heapq.heappop(self.sell_orders)
        self.best_ask = self.sell_orders[0][0] if self.sell_orders else 0

    def get_best_bid_ask(self):
        return self.best_bid, self.best_ask


def process_input(order_book, action):
    action_parts = action.split()

    if not action_parts:
        return

    command = action_parts[0]

    if command == 'O':
        best_bid, best_ask = order_book.get_best_bid_ask()
        print(f"({best_bid}, {best_ask})")
    elif command == 'C':
        if len(action_parts) != 2:
            print("Invalid cancel command. Usage: C [Order ID]")
        else:
            order_id = action_parts[1]
            order_book.cancel_order(order_id)
    elif command in ['B', 'S']:
        if len(action_parts) != 4:
            print("Invalid order command. Usage: [Side][Order ID][Quantity][Price]")
        else:
            side = action_parts[0]
            order_id = action_parts[1]
            try:
                quantity = float(action_parts[2])
                price = float(action_parts[3])
                order_book.submit_order(side, order_id, quantity, price)
            except ValueError:
                print("Invalid quantity or price. Please provide numeric values.")
    else:
        print("Invalid command.")


def main():
    order_book = OrderBook()

    for line in sys.stdin:
        action = line.strip()
        if action == 'END':
            break
        process_input(order_book, action)


if __name__ == '__main__':
    main()
